In [81]:
# !pip install bitsandbytes accelerate

In [82]:
import pandas as pd
import numpy as np

In [84]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Подготовка данных

In [85]:
reviews_data = pd.read_csv("C:\\Users\\just_\\Downloads\\archive\\Books_rating.csv", nrows=10)
reviews_data.head(3)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."


In [86]:
grouped_reviews = reviews_data.groupby('Id')['review/text'].apply(lambda x: list([f'Review {i + 1}: ' + v for i, v in enumerate(x)])).reset_index()
grouped_reviews.head(3)

,Id,review/text
0,826414346,[Review 1: I don't care much for Dr. Seuss but...
1,1882931173,[Review 1: This is only for Julie Strain fans....


In [87]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [88]:
input_text =  f"""Your task is to summarize book reviews and to exctract pros and cons. 
Example:
Review: I didn't like this book, because it has so stupid characters. But the world in the book is great.
Review: Amazing! It's very intersting and breathtaking.
Review: The book is very good, plot especially. But Sam takes too much attention to him.
Summary: Interesting plot, bad characters, great world

{''.join(grouped_reviews['review/text'].iloc[1])}
Summary:
"""
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, 
                            max_length=1024,
                            min_length=8,
                            top_k=150,
                            top_p=0.9,
#                             num_beams=10,o
#                             no_repeat_ngram_size=3,
                            temperature=0.2,
#                             repetition_penalty=2.1,
                            do_sample=True
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["This is a collection of Julie Strain's photos, but it's not a book for everyone."]


In [89]:
print(input_text)

Your task is to summarize book reviews and to exctract pros and cons. 
Example:
Review: I didn't like this book, because it has so stupid characters. But the world in the book is great.
Review: Amazing! It's very intersting and breathtaking.
Review: The book is very good, plot especially. But Sam takes too much attention to him.
Summary: Interesting plot, bad characters, great world

Review 1: This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paintings by Olivia.If you're looking for heavy literary content, this isn't the place to find it -- there's only about 2 pages with text and everything else is photos.Bottom line: if you only want one book, the Six Foot One ... is probably a better choice, however, if you like Julie like I like Julie, you won't go wrong on this one either.
Summary:



In [90]:
from transformers import pipeline
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY", )

In [94]:
input_text =  f"""
{''.join(grouped_reviews['review/text'].iloc[1])[:1024]}
"""
summarizer(input_text)

[{'summary_text': "Julie Strain fans will enjoy this book. It's a collection of her photos with paintings by Olivia."}]